In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string

In [7]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

In [8]:
fake.head()


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [9]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [10]:
fake['class']=0
true['class']=1

In [11]:
data = pd.concat([fake,true], axis=0)

In [12]:
data.sample(10)

,title,text,subject,date,class
6324,Big security risks in Trump feud with spy agen...,WASHINGTON (Reuters) - An unprecedented pre-pr...,politicsNews,"January 13, 2017",1
11660,TOMI LAHREN Blasts The Left For Attacking Trum...,https://www.youtube.com/watch?time_continue=1&...,politics,"Feb 14, 2017",0
16692,"UK certain Iran nuclear deal to be preserved, ...",LONDON (Reuters) - There is absolutely no doub...,worldnews,"October 23, 2017",1
1732,Trump says confident Congress will solve Dream...,MEXICO CITY (Reuters) - U.S. President Donald ...,politicsNews,"September 14, 2017",1
18874,WATCH: HILARIOUS Video Proves CNN Doesn’t Even...,Watch these hilarious examples of CNN having r...,left-news,"Apr 3, 2017",0
17205,"Catalonia refuses to renounce independence, se...",BARCELONA (Reuters) - Catalonia refused on Tue...,worldnews,"October 17, 2017",1
13260,Turkey's Erdogan says will not succumb to U.S....,ANKARA (Reuters) - Turkish President Erdogan s...,worldnews,"December 3, 2017",1
21382,MIND-BLOWING INTERACTIVE MAP Shows Where Musli...,This is a great visual to share with people wh...,left-news,"Oct 29, 2015",0
19198,LIBERAL FED JUDGE Who Sided With BLACK LIVES M...,"First, U.S. District Judge Nathaniel M. Gorton...",left-news,"Feb 4, 2017",0
7181,WATCH: Cartoon Trump Has Something To Say Abo...,Donald Trump isn t the guy you would think wou...,News,"March 31, 2016",0


In [13]:
data = data.drop(["title","subject","date"], axis=1)

In [14]:
data.reset_index(inplace=True)

In [15]:
data.drop(['index'],axis=1 , inplace=True)

In [16]:
data.sample(5)

,text,class
28321,WASHINGTON (Reuters) - The U.S. House of Repre...,1
26011,MOSCOW (Reuters) - Cooperation between Russia ...,1
26108,WASHINGTON (Reuters) - President Donald Trump ...,1
20054,"It s easy to blame the White man that way, y...",0
4341,While the amount of Trump s ahem charitable gi...,0


In [17]:
import re
import string

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', "", text)
    text = re.sub(r'\W', " ", text)
    text = re.sub(r'https?://\S+|www\.\S+', "", text)
    text = re.sub(r'<.*?>+', "", text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), "", text)
    text = re.sub(r'\n', "", text)
    text = re.sub(r'\w*\d\w*', "", text)
    return text


In [18]:
data["text"] = data["text"].apply(clean_text)

In [19]:
x=data["text"]
y=data["class"]

xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.25,random_state=42)

In [20]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [21]:
lr =  LogisticRegression()
lr.fit(xv_train,ytrain)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [22]:
prediciton = lr.predict(xv_test)
lr.score(xv_test,ytest)

0.9858351893095768

In [23]:
print(classification_report(ytest,prediciton))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [24]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']